In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
sys.path.insert(0, '../src')

from options_toolkit.bsm import bsm_call_price, bsm_put_price, implied_volatility
from options_toolkit.viz import payoff_heatmap
from ipywidgets import interact, FloatSlider, Dropdown

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

print("✅ Implied Vol toolkit loaded! Interactive widgets ready.")


✅ Implied Vol toolkit loaded! Interactive widgets ready.


## 1. Option Price vs Volatility

First, let's see how call option price varies with volatility σ.

**Key insight**: Option prices are monotonically increasing in volatility. Higher volatility means more uncertainty, which increases option value (both calls and puts).


In [8]:
# 🎛️ INTERACTIVE: Option Price vs Volatility

@interact(
    S0=FloatSlider(value=100, min=80, max=120, step=5, description='Stock ($):'),
    K=FloatSlider(value=100, min=80, max=120, step=5, description='Strike ($):'),
    r=FloatSlider(value=0.05, min=0.0, max=0.15, step=0.01, description='Rate:'),
    T=FloatSlider(value=1.0, min=0.25, max=2.0, step=0.25, description='Time (yrs):')
)
def plot_price_vs_vol(S0=100, K=100, r=0.05, T=1.0):
    """Interactive plot of option price vs volatility."""
    sigma_range = np.linspace(0.05, 0.60, 100)
    call_prices = [bsm_call_price(S0, K, r, sigma, T) for sigma in sigma_range]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(sigma_range * 100, call_prices, 'b-', linewidth=2.5)
    ax.axvline(x=20, color='r', linestyle='--', linewidth=2, alpha=0.5, label='σ = 20%')
    ax.set_xlabel('Volatility (%)', fontsize=12)
    ax.set_ylabel('Call Option Price ($)', fontsize=12)
    ax.set_title(f'Call Price vs Volatility (S=${S0}, K=${K})', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=11)
    plt.tight_layout()
    plt.show()
    
    print(f"At σ=10%: ${bsm_call_price(S0, K, r, 0.10, T):.4f}")
    print(f"At σ=20%: ${bsm_call_price(S0, K, r, 0.20, T):.4f}")
    print(f"At σ=40%: ${bsm_call_price(S0, K, r, 0.40, T):.4f}")


interactive(children=(FloatSlider(value=100.0, description='Stock ($):', max=120.0, min=80.0, step=5.0), Float…

In [9]:
# 🎛️ INTERACTIVE IMPLIED VOLATILITY SOLVER

@interact(
    market_price=FloatSlider(value=10, min=1, max=50, step=0.5, description='Market Price ($):'),
    S0=FloatSlider(value=100, min=80, max=120, step=5, description='Stock ($):'),
    K=FloatSlider(value=100, min=80, max=120, step=5, description='Strike ($):'),
    r=FloatSlider(value=0.05, min=0.0, max=0.15, step=0.01, description='Rate:'),
    T=FloatSlider(value=1.0, min=0.25, max=2.0, step=0.25, description='Time (yrs):'),
    option_type=Dropdown(options=['call', 'put'], value='call', description='Option:')
)
def solve_iv(market_price=10, S0=100, K=100, r=0.05, T=1.0, option_type='call'):
    """Interactive IV solver."""
    try:
        iv = implied_volatility(market_price, S0, K, r, T, option_type=option_type)
        
        # Verify by pricing with recovered IV
        if option_type == 'call':
            check_price = bsm_call_price(S0, K, r, iv, T)
        else:
            check_price = bsm_put_price(S0, K, r, iv, T)
        
        print("═" * 60)
        print(f"📊 IMPLIED VOLATILITY SOLUTION")
        print("═" * 60)
        print(f"Market Price:      ${market_price:.4f}")
        print(f"Implied Vol (IV):  {iv*100:.2f}%")
        print(f"Verification:      ${check_price:.4f}")
        print(f"Error:             ${abs(check_price - market_price):.6f}")
        print("═" * 60)
        print(f"\n✅ The market is pricing {option_type} at {iv*100:.2f}% annualized volatility")
        
    except ValueError as e:
        print(f"❌ Error: {e}")
        print(f"\nTip: Market price must be > intrinsic value")


interactive(children=(FloatSlider(value=10.0, description='Market Price ($):', max=50.0, min=1.0, step=0.5), F…

In [10]:
low_iv = 0.15
high_iv = 0.35

# Compute prices for different strikes
strikes = np.array([90, 95, 100, 105, 110])
low_iv_prices = [bsm_call_price(S0, K_val, r, low_iv, T) for K_val in strikes]
high_iv_prices = [bsm_call_price(S0, K_val, r, high_iv, T) for K_val in strikes]

# Create comparison table
df = pd.DataFrame({
    'Strike': strikes,
    f'Call Price (IV={low_iv*100:.0f}%)': low_iv_prices,
    f'Call Price (IV={high_iv*100:.0f}%)': high_iv_prices,
    'Difference': np.array(high_iv_prices) - np.array(low_iv_prices)
})

print(df.to_string(index=False))
print(f"\nObservation: High IV environment has much higher option premiums!")
print(f"ATM call: ${low_iv_prices[2]:.2f} (low IV) vs ${high_iv_prices[2]:.2f} (high IV)")


 Strike  Call Price (IV=15%)  Call Price (IV=35%)  Difference
     90            15.467159            21.321119    5.853960
     95            11.753845            18.583271    6.829426
    100             8.591658            16.128429    7.536771
    105             6.035564            13.944175    7.908612
    110             4.075866            12.014090    7.938224

Observation: High IV environment has much higher option premiums!
ATM call: $8.59 (low IV) vs $16.13 (high IV)


## 4. Heatmap: Call Price vs Stock Price and Volatility

Let's create a 2D heatmap showing how call option price varies with both underlying price and volatility.


In [11]:
# 🎛️ INTERACTIVE VOLATILITY SURFACE HEATMAP

@interact(
    K=FloatSlider(value=100, min=80, max=120, step=10, description='Strike ($):'),
    r=FloatSlider(value=0.05, min=0.0, max=0.15, step=0.01, description='Rate:'),
    T=FloatSlider(value=1.0, min=0.25, max=2.0, step=0.25, description='Time (yrs):')
)
def plot_vol_surface(K=100, r=0.05, T=1.0):
    """Interactive volatility surface."""
    S_vals = np.linspace(K*0.7, K*1.3, 50)
    sigma_vals = np.linspace(0.10, 0.50, 40)
    
    prices = np.zeros((len(sigma_vals), len(S_vals)))
    for i, sigma in enumerate(sigma_vals):
        for j, S in enumerate(S_vals):
            prices[i, j] = bsm_call_price(S, K, r, sigma, T)
    
    fig = payoff_heatmap(S_vals, sigma_vals, prices,
                         xlabel="Stock Price ($)",
                         ylabel="Volatility (σ)",
                         title=f"Call Option Price Surface (K=${K}, T={T}yr)",
                         cmap="viridis")
    plt.show()
    
    print(f"📊 Heatmap shows option value across stock prices and volatilities")
    print(f"   Bright regions = High option value | Dark regions = Low value")


interactive(children=(FloatSlider(value=100.0, description='Strike ($):', max=120.0, min=80.0, step=10.0), Flo…